In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import hipp
import os
import usgsxplore
from pathlib import Path

## Settings

In [5]:
# Paths
PROJECT_DIRECTORY = Path("/home/godinlu/github/hipp/notebooks/data/kh9pc")
RAW_IMAGES = PROJECT_DIRECTORY / "raw_images"
JOINED_IMAGES = PROJECT_DIRECTORY / "joined_images"
PREPROCESSED_IMAGES = PROJECT_DIRECTORY / "preprocessed_images"
QC_DIRECTORY = PROJECT_DIRECTORY / "qc_dir"

QUICKVIEW_FACTOR = 0.05
MAX_WORKERS = 4
OVERWRITE = False

## Step 1 : Download raw images

To download the raw images we will use [`usgsxplore`](https://github.com/adehecq/usgs_explorer) which is an python interface to the [USGS M2M API](https://m2m.cr.usgs.gov/) to search and download data available from the [Earth Explorer](https://earthexplorer.usgs.gov/) platform.
We will download 2 images and each image is separated in 10 or 12 parts : `_a`, `_b`, `_c`, ...
The total downloading size is 16 Go, so it can take a will. Further more, the stagging links from the [USGS M2M API](https://m2m.cr.usgs.gov/) can take some time.
The downloading include already the extracting of tif files.

In [ ]:
username = os.getenv("USGS_USERNAME") or input("Enter your USGS username: ")
token = os.getenv("USGS_TOKEN") or input("Enter your USGS token: ")

entity_ids = ["D3C1214-100097A014", "D3C1214-100097A015"]

api = usgsxplore.API(username, token)
api.download("declassiii", entity_ids, output_dir=RAW_IMAGES)
api.logout()

In [ ]:
hipp.tools.optimize_geotifs(RAW_IMAGES)
hipp.tools.generate_quickviews(RAW_IMAGES, QUICKVIEW_FACTOR, max_workers=8)

## Step 2 : Joining Images

The first step of the preprocessing pipeline is to **join multiple image tiles** into a single, continuous image. This process is necessary because **KH9 PC images are typically split into 10-12 separate parts**, each approximately **1 GB in size**, due to their large original resolution.

However, **joining these image parts is not straightforward**, primarily because there is a **slight overlap between adjacent tiles**. 

To accurately reconstruct the full image, we need to perform the following steps:

1. **Detect keypoints (interest points)** along the **right border** of the first image part.
2. Detect corresponding keypoints along the **left border** of the next image part.
3. **Match these keypoints** using a feature-matching algorithm to find candidate correspondences.
4. Use **RANSAC (Random Sample Consensus)** to filter out mismatched or erroneous correspondences (i.e., outliers).
5. Estimate a **relative geometric transformation** (typically a translation or affine transform) to correctly align the second image with the first.
6. Apply this transformation and **merge the two parts** into a larger composite image.

This process is repeated sequentially for all image parts, progressively building up the full image mosaic.

To perform that 2 functions exists:
- `join_images_asp` : which will use [`image_mosaic`](https://stereopipeline.readthedocs.io/en/latest/tools/image_mosaic.html) program from Ames Stereo Pipeline. This function is much more safer but it requiered to install ASP and the command need to be visible in your path.
- `join_images` : is a pure Python implementation that replicates the same processing steps, it's faster an generate some qc plots. (8 min vs 40 min for asp)


In [ ]:
#hipp.kh9pc.join_images_asp(RAW_IMAGES, JOINED_IMAGES)

hipp.kh9pc.join_images(RAW_IMAGES, JOINED_IMAGES, OVERWRITE, max_workers=MAX_WORKERS)

hipp.tools.generate_quickviews(JOINED_IMAGES, QUICKVIEW_FACTOR, max_workers=MAX_WORKERS, overwrite=OVERWRITE)

## Step 3: Processing the Joined Image Using Collimation Lines

The goal of this step is to correct distortions and crop the image based on the top and bottom collimation lines. This process is essential because the joined image contains both a region of interest (ROI) and a background area. The ROI may not be perfectly centered or aligned within the background, so we aim to isolate and retain only the ROI.

This step involves the following sub-processes:

- **Estimate both collimation lines using a polynomial model:**
  - Vertically scan the image to detect intensity peaks.
  - For each column, keep the peak with the highest prominence.
  - Fit a second-degree polynomial using RANSAC on the detected peaks.

- **Estimate the vertical edges (x₁ and x₂) of the ROI:**
  - Horizontally scan the image and record points where intensity values exceed a defined background threshold.
  - Fit constant lines using RANSAC to estimate x₁ and x₂.

- **Create source and target grid points:**
  - Generate the source points by sampling along both detected collimation lines, forming a grid between them.
  - Generate the target points by sampling along the theoretical collimation lines (spaced by 21,770 px) and building a corresponding grid.

- **Remap the image using Thin Plate Spline (TPS):**
  - Compute the TPS transformation using the source and target points, then apply it to the image.
  - To reduce computation time, calculate the transformation coordinates at a lower resolution with TPS and upscale them using a bivariate spline interpolation for the full resolution.


This all pipeline is based on collimations lines detection. And it can appends to detect wrong lines with wrong padding, so make sure both collimation lines are correct before next steps.

In [21]:
hipp.kh9pc.iter_collimation_rectification(JOINED_IMAGES, PREPROCESSED_IMAGES, QC_DIRECTORY)

Collimation rectification for D3C1214-100097A014.tif : 
	-[1/4] Estimation of collimation lines...
	-[2/4] Detection of vertical lines...
	-[3/4] Warping image (can take some times)...


D3C1214-100097A014.tif remapping: 100%|████████████████████████████████████████████████████████████████████████████████████████| 129/129 [14:35<00:00,  6.79s/block]


	-[4/4] Estimation of collimation lines after transformation...
Collimation rectification for D3C1214-100097A015.tif : 
	-[1/4] Estimation of collimation lines...
	-[2/4] Detection of vertical lines...
	-[3/4] Warping image (can take some times)...


D3C1214-100097A015.tif remapping: 100%|████████████████████████████████████████████████████████████████████████████████████████| 126/126 [17:25<00:00,  8.30s/block]


	-[4/4] Estimation of collimation lines after transformation...
